In [ ]:
# 필수 패키지 설치
!pip install -q music21 pydub midi2audio

# 시스템 패키지 설치
!sudo apt-get update
!sudo apt-get install -y ffmpeg fluidsynth

# 디렉토리 생성
import os
os.makedirs("output_midi", exist_ok=True)
os.makedirs("output_mp3", exist_ok=True)

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,246 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,742 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packa

In [ ]:
emotion_music_config = {
    "불평/불만": {"key": "A minor", "tempo": 90, "notes": ["A4", "C5", "B4", "G4", "F4", "E4"]},
    "환영/호의": {"key": "C major", "tempo": 100, "notes": ["C5", "E5", "G5", "C6"]},
    "감동/감탄": {"key": "D major", "tempo": 80, "notes": ["D5", "F#5", "A5", "B5"]},
    "지긋지긋": {"key": "B minor", "tempo": 60, "notes": ["B4", "D5", "A4", "F#4"]},
    "고마움": {"key": "E major", "tempo": 95, "notes": ["E5", "G#5", "B5"]},
    "슬픔": {"key": "D minor", "tempo": 65, "notes": ["D4", "F4", "A4", "C4"]},
    "화남/분노": {"key": "C minor", "tempo": 110, "notes": ["C4", "G4", "Bb4", "Eb4"]},
    "존경": {"key": "F major", "tempo": 85, "notes": ["F4", "A4", "C5", "E5"]},
    "기대감": {"key": "G major", "tempo": 100, "notes": ["G4", "B4", "D5", "E5"]},
    "우쭐댐/무시함": {"key": "E minor", "tempo": 105, "notes": ["E4", "G4", "B4", "D5"]},
    "안타까움/실망": {"key": "A minor", "tempo": 70, "notes": ["A3", "C4", "G3"]},
    "비장함": {"key": "F# minor", "tempo": 90, "notes": ["F#4", "A4", "C#5", "E5"]},
    "의심/불신": {"key": "C minor", "tempo": 60, "notes": ["C4", "Eb4", "G4", "Ab4"]},
    "뿌듯함": {"key": "D major", "tempo": 100, "notes": ["D4", "F#4", "A4", "B4"]},
    "편안/쾌적": {"key": "F major", "tempo": 75, "notes": ["F4", "A4", "C5"]},
    "신기함/관심": {"key": "G major", "tempo": 95, "notes": ["G4", "B4", "D5", "F#5"]},
    "아껴주는": {"key": "C major", "tempo": 80, "notes": ["C4", "E4", "G4"]},
    "부끄러움": {"key": "A minor", "tempo": 65, "notes": ["A4", "C5", "D5"]},
    "공포/무서움": {"key": "D minor", "tempo": 50, "notes": ["D3", "F3", "A3", "C4"]},
    "절망": {"key": "B minor", "tempo": 55, "notes": ["B3", "D4", "F#4", "A4"]},
    "한심함": {"key": "C minor", "tempo": 60, "notes": ["C3", "Eb3", "G3"]},
    "역겨움/징그러움": {"key": "E minor", "tempo": 70, "notes": ["E3", "G3", "Bb3"]},
    "짜증": {"key": "G minor", "tempo": 100, "notes": ["G3", "Bb3", "D4"]},
    "어이없음": {"key": "D major", "tempo": 90, "notes": ["D4", "F#4", "A4", "C#5"]},
    "없음": {"key": "C major", "tempo": 90, "notes": ["C4", "E4", "G4"]},
    "패배/자기혐오": {"key": "A minor", "tempo": 60, "notes": ["A3", "C4", "D4"]},
    "귀찮음": {"key": "B minor", "tempo": 70, "notes": ["B3", "D4", "A3"]},
    "힘듦/지침": {"key": "E minor", "tempo": 65, "notes": ["E3", "G3", "B3"]},
    "즐거움/신남": {"key": "C major", "tempo": 120, "notes": ["C5", "E5", "G5", "A5"]},
    "깨달음": {"key": "F major", "tempo": 85, "notes": ["F4", "A4", "C5", "D5"]},
    "죄책감": {"key": "D minor", "tempo": 55, "notes": ["D3", "F3", "A3", "Bb3"]},
    "증오/혐오": {"key": "G minor", "tempo": 90, "notes": ["G3", "Bb3", "D4", "F4"]},
    "흐뭇함(귀여움/예쁨)": {"key": "C major", "tempo": 95, "notes": ["C5", "E5", "G5", "B5"]},
    "당황/난처": {"key": "F# minor", "tempo": 80, "notes": ["F#4", "A4", "C#5"]},
    "경악": {"key": "E minor", "tempo": 110, "notes": ["E4", "G4", "B4", "C5"]},
    "부담/안_내킴": {"key": "C minor", "tempo": 70, "notes": ["C3", "Eb3", "G3"]},
    "서러움": {"key": "A minor", "tempo": 60, "notes": ["A3", "C4", "E4", "D4"]},
    "재미없음": {"key": "D minor", "tempo": 60, "notes": ["D3", "F3", "A3"]},
    "불쌍함/연민": {"key": "G minor", "tempo": 60, "notes": ["G3", "Bb3", "D4"]},
    "놀람": {"key": "C major", "tempo": 120, "notes": ["C4", "E4", "G4", "B4"]},
    "행복": {"key": "D major", "tempo": 110, "notes": ["D5", "F#5", "A5", "C#6"]},
    "불안/걱정": {"key": "E minor", "tempo": 70, "notes": ["E3", "G3", "B3", "D4"]},
    "기쁨": {"key": "G major", "tempo": 115, "notes": ["G4", "B4", "D5", "F#5"]},
    "안심/신뢰": {"key": "F major", "tempo": 85, "notes": ["F4", "A4", "C5", "E5"]}
}

In [ ]:
emotion_instruments = {
    "불평/불만": "Cello",
    "환영/호의": "Acoustic Guitar",
    "감동/감탄": "French Horn",
    "지긋지긋": "Bassoon",
    "고마움": "Oboe",
    "슬픔": "Violin",
    "화남/분노": "Trombone",
    "존경": "Trumpet",
    "기대감": "Clarinet",
    "우쭐댐/무시함": "Saxophone",
    "안타까움/실망": "Bass Clarinet",
    "비장함": "Timpani",
    "의심/불신": "Contrabass",
    "뿌듯함": "Marimba",
    "편안/쾌적": "Harp",
    "신기함/관심": "Celesta",
    "아껴주는": "Flute",
    "부끄러움": "English Horn",
    "공포/무서움": "Church Organ",
    "절망": "Bassoon",
    "한심함": "Muted Trumpet",
    "역겨움/징그러움": "Alto Flute",
    "짜증": "Electric Guitar",
    "어이없음": "Xylophone",
    "없음": "Piano",
    "패배/자기혐오": "Tuba",
    "귀찮음": "Bass Trombone",
    "힘듦/지침": "Harmonica",
    "즐거움/신남": "Ukulele",
    "깨달음": "Recorder",
    "죄책감": "Muted Violin",
    "증오/혐오": "Distortion Guitar",
    "흐뭇함(귀여움/예쁨)": "Music Box",
    "당황/난처": "Kalimba",
    "경악": "Synthesizer",
    "부담/안_내킴": "Oboe",
    "서러움": "Viola",
    "재미없음": "Monotone Piano",
    "불쌍함/연민": "Accordion",
    "놀람": "Glockenspiel",
    "행복": "Acoustic Piano",
    "불안/걱정": "Bass Flute",
    "기쁨": "Pan Flute",
    "안심/신뢰": "Clarinet"
}

In [ ]:
import os

os.makedirs('output_midi', exist_ok=True)
os.makedirs('output_mp3', exist_ok=True)

In [ ]:
from music21 import stream, note, tempo, key
from midi2audio import FluidSynth
from music21 import instrument  # 추가

def create_midi(emotion, config, duration=10):
    s = stream.Stream()
    s.append(tempo.MetronomeMark(number=config['tempo']))
    key_sig = key.Key(config['key'].split()[0], config['key'].split()[1])
    s.append(key_sig)

    # 감정에 따른 악기 설정
    instr_name = emotion_instruments.get(emotion, "Piano")
    instr_class = getattr(instrument, instr_name.replace(" ", ""), instrument.Piano)
    s.append(instr_class())

    seconds_per_beat = 60 / config['tempo']
    beats_needed = int(duration / seconds_per_beat)

    for i in range(beats_needed):
        n = note.Note(config['notes'][i % len(config['notes'])])
        n.quarterLength = 1
        s.append(n)

    midi_path = f'output_midi/{emotion}.mid'
    s.write('midi', fp=midi_path)
    return midi_path

def convert_to_mp3(midi_path, mp3_path):
    fs = FluidSynth()
    fs.midi_to_audio(midi_path, mp3_path)

In [ ]:
def create_midi_varied(emotion, config, duration=10):
    from music21 import stream, tempo, key, instrument, chord

    s = stream.Stream()
    s.append(tempo.MetronomeMark(number=config['tempo']))
    key_sig = key.Key(config['key'].split()[0], config['key'].split()[1])
    s.append(key_sig)

    instr_name = emotion_instruments.get(emotion, "Piano")
    instr_class = getattr(instrument, instr_name.replace(" ", ""), instrument.Piano)
    s.append(instr_class())

    seconds_per_beat = 60 / config['tempo']
    beats_needed = int(duration / seconds_per_beat)

    rhythm_pattern = [1, 0.5, 0.5, 1.5]

    notes = config['notes']
    notes_len = len(notes)

    # 화음 노트 리스트 만들기 (노트 개수에 맞게)
    if notes_len >= 6:
        chord_notes = [
            [notes[0], notes[2], notes[4]],
            [notes[1], notes[3], notes[5]],
        ]
    elif notes_len >= 4:
        chord_notes = [
            [notes[0], notes[2]],
            [notes[1], notes[3]],
        ]
    else:
        chord_notes = [[notes[0]]]  # 단일 음

    for i in range(beats_needed):
        current_chord_notes = chord_notes[i % len(chord_notes)]
        if len(current_chord_notes) > 1:
            c = chord.Chord(current_chord_notes)
        else:
            from music21 import note
            c = note.Note(current_chord_notes[0])

        c.quarterLength = rhythm_pattern[i % len(rhythm_pattern)]

        # 옥타브 변주
        octave_shift = (i % 2)
        if isinstance(c, chord.Chord):
            for n in c.notes:
                n.octave += octave_shift
        else:
            c.octave += octave_shift

        s.append(c)

    midi_path = f'output_midi/{emotion}.mid'
    s.write('midi', fp=midi_path)
    return midi_path

In [ ]:
import random
from music21 import stream, tempo, key, instrument, note

def create_midi_varied_v2(emotion, config, duration=10):
    s = stream.Stream()
    s.append(tempo.MetronomeMark(number=config['tempo']))

    key_root, key_mode = config['key'].split()
    key_sig = key.Key(key_root, key_mode)
    s.append(key_sig)

    instr_name = config.get('instrument', 'Piano')
    instr_class = getattr(instrument, instr_name.replace(" ", ""), instrument.Piano)
    s.append(instr_class())

    seconds_per_beat = 60 / config['tempo']
    beats_needed = int(duration / seconds_per_beat)

    base_notes = config['notes']

    for i in range(beats_needed):
        base_note = random.choice(base_notes)
        n = note.Note(base_note)

        # 옥타브 변주 (-1,0,1)
        n.octave += random.choice([-1, 0, 1])

        # 박자 변주 (0.5, 1, 1.5)
        n.quarterLength = random.choice([0.5, 1, 1.5])

        s.append(n)

    midi_path = f'output_midi/{emotion}.mid'
    s.write('midi', fp=midi_path)
    return midi_path

In [ ]:
def sanitize_filename(name):
    return name.replace('/', '_')  # 슬래시를 밑줄(_)로 대체

for emotion, config in emotion_music_config.items():
    print(f"{emotion} → 생성 중...")
    safe_name = sanitize_filename(emotion)
    midi_path = create_midi_varied_v2(safe_name, config)  # 감정 이름 대신 safe_name 사용
    mp3 = f"output_mp3/{safe_name}.mp3"
    convert_to_mp3(midi_path, mp3)
print("✅ 전체 MP3 생성 완료!")

불평/불만 → 생성 중...
환영/호의 → 생성 중...
감동/감탄 → 생성 중...
지긋지긋 → 생성 중...
고마움 → 생성 중...
슬픔 → 생성 중...
화남/분노 → 생성 중...
존경 → 생성 중...
기대감 → 생성 중...
우쭐댐/무시함 → 생성 중...
안타까움/실망 → 생성 중...
비장함 → 생성 중...
의심/불신 → 생성 중...
뿌듯함 → 생성 중...
편안/쾌적 → 생성 중...
신기함/관심 → 생성 중...
아껴주는 → 생성 중...
부끄러움 → 생성 중...
공포/무서움 → 생성 중...
절망 → 생성 중...
한심함 → 생성 중...
역겨움/징그러움 → 생성 중...
짜증 → 생성 중...
어이없음 → 생성 중...
없음 → 생성 중...
패배/자기혐오 → 생성 중...
귀찮음 → 생성 중...
힘듦/지침 → 생성 중...
즐거움/신남 → 생성 중...
깨달음 → 생성 중...
죄책감 → 생성 중...
증오/혐오 → 생성 중...
흐뭇함(귀여움/예쁨) → 생성 중...
당황/난처 → 생성 중...
경악 → 생성 중...
부담/안_내킴 → 생성 중...
서러움 → 생성 중...
재미없음 → 생성 중...
불쌍함/연민 → 생성 중...
놀람 → 생성 중...
행복 → 생성 중...
불안/걱정 → 생성 중...
기쁨 → 생성 중...
안심/신뢰 → 생성 중...
✅ 전체 MP3 생성 완료!
